In [1]:
import cv2
import pandas as pd
import re
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score
from keras.models import load_model
from pathlib import Path

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-01-05 12:37:56.359340: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
TIMEFRAMES = [14, 30, 90, 180, 365]
PREDICTIONS = [5, 30, 90]
IMG_TYPES = ['OHLC', 'ColoredOHLC', 'Line', 'AlgoTrading']
NEW_DATA = ['SmallCap', 'Russell2000', 'Treasury', 'Silver', 'Semiconductor', 'Healthcare']

labels = pd.read_csv('transfer_labels.csv')

In [19]:
# Crop the images using the bounding boxes
def crop_image(img_path):
    # Load the image in grayscale
    img = cv2.imread(img_path, 0)

    # Check if the image was loaded correctly
    if img is None:
        raise ValueError(f"Image at {img_path} not found. Please check the path.")

    # Use regular expression to match numbers followed by ".png" at the end of the filename
    match = re.search(r'(\d+)(?=\.png$)', img_path)
    
    # Check if we found a match
    if match:
        # Extract the number from the matched group
        number = int(match.group(1))
        
        # Check if the number is one of the specified values
        if number == 14:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+85]
        elif number == 30:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+132]
        elif number == 90:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+226]
        elif number == 180:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+414]
        elif number == 365:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+602]
    return crop

# Example usage:
filenames = labels['Image'].values.tolist()

# Testing the function with the provided list of filenames
for name in filenames:
    try:
        cropped_image = crop_image(name)
        # Construct the new path for the cropped image
        new_path = name.replace('.png', '_cropped.png')
        # Save the cropped image
        cv2.imwrite(new_path, cropped_image)
    except ValueError as e:
        print(e)

In [3]:
# Create a new column called 'Image' that contains the path to the cropped image but only if they 
labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')

/var/folders/jx/r9zzxsjd7wxgmmn32r0l8cz40000gn/T/ipykernel_99604/666818149.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')


In [4]:
# Function to load and convert an image to grayscale
def load_image(image_path):
    # Load image in grayscale
    image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Unable to load image at path: {image_path}")
    return image

filenames = labels['Image'].values.tolist()

images = []

for name in filenames:
    try:
        img = load_image(name)
        images.append(img)
    except ValueError as e:
        print(e)

# Add a new column to the labels DataFrame to store the image arrays
labels['Image_Array'] = images

In [5]:
# Show the number of 1s and 0s in the dataset
print(labels['Label'].value_counts())

1    4580
0    3220
Name: Label, dtype: int64


In [6]:
# Sort the DataFrame by date
labels['Date'] = labels['Image'].str.extract(r'(\d{4}-\d{2}-\d{2})')
labels['Date'] = pd.to_datetime(labels['Date'])
labels = labels.sort_values(by='Date') 
print(labels.head())

                                                  Image  TimePrediction  \
0     images/OHLC/SmallCap_2019-02-01 00:00:00_14_cr...               5   
2630  images/Line/Treasury_2019-02-01 00:00:00_30_cr...               5   
2600  images/Line/Treasury_2019-02-01 00:00:00_14_cr...               5   
2530  images/Line/Russell2000_2019-02-01 00:00:00_36...              90   
2529  images/Line/Russell2000_2019-02-01 00:00:00_36...              30   

       LastPrice  FuturePrice  Label  \
0      76.620003    76.860001      1   
2630  120.959999   121.830002      1   
2600  120.959999   121.830002      1   
2530  149.179993   160.529999      1   
2529  149.179993   156.809998      1   

                                            Image_Array       Date  
0     [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,... 2019-02-01  
2630  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,... 2019-02-01  
2600  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,... 2019-02-01  
2530  [[0, 0, 0, 0, 0, 15, 15, 1

In [7]:
evaluation_df = pd.DataFrame(columns=['Image_Type', 'Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Long_Average_RoR', 'Long_Investment', 'Long_Return', 'Short_Average_RoR', 'Short_Investment', 'Short_Return', 'Total_Investment', 'Total_Return'])

TIMEFRAMES = [14, 30, 90, 180, 365]
PREDICTIONS = [5, 30, 90]
#IMG_TYPES = ['OHLC', 'ColoredOHLC']
IMG_TYPES = ['Line', 'AlgoTrading']

for img_type in IMG_TYPES:
    for timeframe in TIMEFRAMES:
        for prediction in PREDICTIONS:
            if prediction < timeframe:
                print(f"Evaluating model predicting {prediction} days ahead using {img_type} images with {timeframe} days timeframe.")

                # Filter the data
                data = labels[(labels['TimePrediction'] == prediction) &
                              (labels['Image'].str.contains(img_type)) &
                              (labels['Image'].str.contains(f'_{timeframe}_'))]
                data = data.reset_index(drop=True)

                # Start at same index as test data
                split_index = int(len(data) * 0.7)
                data = data[split_index:]

                # Load the model
                model_filename = f"models/{img_type}_{timeframe}_{prediction}.h5"
                model = load_model(model_filename)

                X = np.array(data['Image_Array'].tolist()) / 255.0
                lastPrice = data['LastPrice'].tolist()
                futurePrice = data['FuturePrice'].tolist()
                y = data['Label'].values

                # Evaluate the model on test data
                y_pred = model.predict(X)
                # Convert predictions to binary: if > 0.5 then 1 else 0
                y_pred_binary = np.where(y_pred > 0.5, 1, 0)  
                
                accuracy = accuracy_score(y, y_pred_binary)
                precision = precision_score(y, y_pred_binary)
                recall = recall_score(y, y_pred_binary)
                f1_score = fbeta_score(y, y_pred_binary, beta=1)

                y_test_array = y.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
                correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
                hit_rate = correct_predictions / len(y_test_array)

                # Calculate Rate of Return RoR for each trade independently
                # Long positions
                long_ror = []
                long_trades = []
                long_investment = 0
                # Short positions
                short_ror = []
                short_trades = []
                short_investment = 0

                y_pred_binary = y_pred_binary.tolist()
                for i in range(len(y_pred_binary)):
                    # Long position
                    if y_pred_binary[i][0] == 1:
                        long_return_i = ((futurePrice[i] - lastPrice[i])/lastPrice[i])
                        long_ror.append(long_return_i)
                        long_trades.append(100*(long_return_i))
                        long_investment += 100
                    # Short position
                    else:
                        short_return_i = ((lastPrice[i] - futurePrice[i])/lastPrice[i])
                        short_ror.append(short_return_i)
                        short_trades.append(100*(short_return_i))
                        short_investment += 100
                
                # Calculate average long RoR
                long_avg_ror = np.mean(long_ror)
                # Sum all long trades
                long_trades = np.sum(long_trades)
                # Calculate average short RoR
                short_avg_ror = np.mean(short_ror)
                # Sum all short trades
                short_trades = np.sum(short_trades)
                # Total investment
                total_investment = long_investment + short_investment
                total_returns = long_trades + short_trades
                
                
                print("Evaluation Metrics:")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1 Score: {f1_score}")
                print(f"Hit Rate: {hit_rate}")
                print(f"Average Long RoR: {long_avg_ror}")
                print(f"Long Investment: {long_investment}")
                print(f"Long Return: {long_trades}")
                print(f"Average Short RoR: {short_avg_ror}")
                print(f"Short Investment: {short_investment}")
                print(f"Short Return: {short_trades}")
                print(f"Total Investment: {total_investment}")
                print(f"Total Return: {total_returns}")

                
                # Add the evaluation metrics to the DataFrame
                evaluation_df = evaluation_df.append({
                    'Image_Type': img_type,
                    'Timeframe': timeframe,
                    'Prediction': prediction,
                    'Accuracy': accuracy,
                    'Precision': precision,
                    'Recall': recall,
                    'F1_Score': f1_score,
                    'Hit_Rate': hit_rate,
                    'Long_Average_RoR': long_avg_ror,
                    'Long_Investment': long_investment,
                    'Long_Return': long_trades,
                    'Short_Average_RoR': short_avg_ror,
                    'Short_Investment': short_investment,
                    'Short_Return': short_trades, 
                    'Total_Investment': total_investment,
                    'Total_Return': total_returns
                }, ignore_index=True)

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('transfer_evaluation_scores.csv', index=False)
print("Evaluation scores saved to 'tansfer_evaluation_scores.csv'.")

Evaluating model predicting 5 days ahead using Line images with 14 days timeframe.
2/2 [==============================] - 0s 87ms/step
Evaluation Metrics:
Accuracy: 0.6909090909090909
Precision: 0.8125
Recall: 0.7027027027027027
F1 Score: 0.7536231884057971
Hit Rate: 0.6909090909090909
Average Long RoR: 0.02152043291720481
Long Investment: 3200
Long Return: 68.8653853350554
Average Short RoR: 0.0023998282565920325
Short Investment: 2300
Short Return: 5.519604990161676
Total Investment: 5500
Total Return: 74.38499032521707
Evaluating model predicting 5 days ahead using Line images with 30 days timeframe.
2/2 [==============================] - 1s 224ms/step
Evaluation Metrics:
Accuracy: 0.5471698113207547
Precision: 0.8076923076923077
Recall: 0.525
F1 Score: 0.6363636363636364
Hit Rate: 0.5471698113207547
Average Long RoR: 0.01684337131708343
Long Investment: 2600
Long Return: 43.79276542441692
Average Short RoR: -0.009291904304228127
Short Investment: 2700
Short Return: -25.088141621415

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


2/2 [==============================] - 4s 1s/step
Evaluation Metrics:
Accuracy: 0.4897959183673469
Precision: 0.4897959183673469
Recall: 1.0
F1 Score: 0.6575342465753424
Hit Rate: 0.4897959183673469
Average Long RoR: 0.011114765828150954
Long Investment: 4900
Long Return: 54.46235255793969
Average Short RoR: nan
Short Investment: 0
Short Return: 0.0
Total Investment: 4900
Total Return: 54.46235255793969
Evaluating model predicting 90 days ahead using AlgoTrading images with 180 days timeframe.


/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


2/2 [==============================] - 4s 1s/step
Evaluation Metrics:
Accuracy: 0.8571428571428571
Precision: 0.8518518518518519
Recall: 0.8846153846153846
F1 Score: 0.8679245283018868
Hit Rate: 0.8571428571428571
Average Long RoR: 0.07856206953953462
Long Investment: 2700
Long Return: 212.11758775674343
Average Short RoR: 0.07782673447534448
Short Investment: 2200
Short Return: 171.21881584575786
Total Investment: 4900
Total Return: 383.33640360250126
Evaluating model predicting 5 days ahead using AlgoTrading images with 365 days timeframe.
2/2 [==============================] - 5s 1s/step
Evaluation Metrics:
Accuracy: 0.5
Precision: 0.5
Recall: 0.9545454545454546
F1 Score: 0.65625
Hit Rate: 0.5
Average Long RoR: -0.0003284043713180592
Long Investment: 4200
Long Return: -1.3792983595358501
Average Short RoR: -0.007742661567886749
Short Investment: 200
Short Return: -1.5485323135773499
Total Investment: 4400
Total Return: -2.9278306731132
Evaluating model predicting 30 days ahead using

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


2/2 [==============================] - 5s 1s/step
Evaluation Metrics:
Accuracy: 0.4090909090909091
Precision: 0.4090909090909091
Recall: 1.0
F1 Score: 0.5806451612903226
Hit Rate: 0.4090909090909091
Average Long RoR: -0.025626362746094963
Long Investment: 4400
Long Return: -112.75599608281782
Average Short RoR: nan
Short Investment: 0
Short Return: 0.0
Total Investment: 4400
Total Return: -112.75599608281782
Evaluation scores saved to 'tansfer_evaluation_scores.csv'.


/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [13]:
evaluation_df = pd.DataFrame(columns=['Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Long_Average_RoR', 'Long_Investment', 'Long_Return', 'Short_Average_RoR', 'Short_Investment', 'Short_Return', 'Total_Investment', 'Total_Return'])

TIMEFRAMES = [14, 30, 90, 180, 365]
PREDICTIONS = [5, 30, 90]
IMG_TYPES = ['OHLC', 'ColoredOHLC', 'Line', 'AlgoTrading']

for timeframe in TIMEFRAMES:
    for prediction in PREDICTIONS:
        if prediction < timeframe:
            print(f"Evaluating model predicting {prediction} days ahead with {timeframe} days timeframe.")

            # Filter the data
            data = labels[(labels['TimePrediction'] == prediction) &
                            (labels['Image'].str.contains(f'_{timeframe}_'))]
            data = data.reset_index(drop=True)

            # Start at same index as test data
            split_index = int(len(data) * 0.7)
            data = data[split_index:]

            # Load the model
            model_filename = f"models/combined_{timeframe}_{prediction}.h5"
            model = load_model(model_filename)

            X = np.array(data['Image_Array'].tolist()) / 255.0
            lastPrice = data['LastPrice'].tolist()
            futurePrice = data['FuturePrice'].tolist()
            y = data['Label'].values

            # Evaluate the model on test data
            y_pred = model.predict(X)
            # Convert predictions to binary: if > 0.5 then 1 else 0
            y_pred_binary = np.where(y_pred > 0.5, 1, 0)  
            
            accuracy = accuracy_score(y, y_pred_binary)
            precision = precision_score(y, y_pred_binary)
            recall = recall_score(y, y_pred_binary)
            f1_score = fbeta_score(y, y_pred_binary, beta=1)

            y_test_array = y.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
            correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
            hit_rate = correct_predictions / len(y_test_array)

            # Calculate Rate of Return RoR for each trade independently
            # Long positions
            long_ror = []
            long_trades = []
            long_investment = 0
            # Short positions
            short_ror = []
            short_trades = []
            short_investment = 0

            y_pred_binary = y_pred_binary.tolist()
            for i in range(len(y_pred_binary)):
                # Long position
                if y_pred_binary[i][0] == 1:
                    long_return_i = ((futurePrice[i] - lastPrice[i])/lastPrice[i])
                    long_ror.append(long_return_i)
                    long_trades.append(100*(long_return_i))
                    long_investment += 100
                # Short position
                else:
                    short_return_i = ((lastPrice[i] - futurePrice[i])/lastPrice[i])
                    short_ror.append(short_return_i)
                    short_trades.append(100*(short_return_i))
                    short_investment += 100
            
            # Calculate average long RoR
            long_avg_ror = np.mean(long_ror)
            # Sum all long trades
            long_trades = np.sum(long_trades)
            # Calculate average short RoR
            short_avg_ror = np.mean(short_ror)
            # Sum all short trades
            short_trades = np.sum(short_trades)
            # Total investment
            total_investment = long_investment + short_investment
            total_returns = long_trades + short_trades
            
            
            print("Evaluation Metrics:")
            print(f"Accuracy: {accuracy}")
            print(f"Precision: {precision}")
            print(f"Recall: {recall}")
            print(f"F1 Score: {f1_score}")
            print(f"Hit Rate: {hit_rate}")
            print(f"Average Long RoR: {long_avg_ror}")
            print(f"Long Investment: {long_investment}")
            print(f"Long Return: {long_trades}")
            print(f"Average Short RoR: {short_avg_ror}")
            print(f"Short Investment: {short_investment}")
            print(f"Short Return: {short_trades}")
            print(f"Total Investment: {total_investment}")
            print(f"Total Return: {total_returns}")

            
            # Add the evaluation metrics to the DataFrame
            evaluation_df = evaluation_df.append({
                'Timeframe': timeframe,
                'Prediction': prediction,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1_Score': f1_score,
                'Hit_Rate': hit_rate,
                'Long_Average_RoR': long_avg_ror,
                'Long_Investment': long_investment,
                'Long_Return': long_trades,
                'Short_Average_RoR': short_avg_ror,
                'Short_Investment': short_investment,
                'Short_Return': short_trades, 
                'Total_Investment': total_investment,
                'Total_Return': total_returns
            }, ignore_index=True)

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('transfer_combined_evaluation_scores.csv', index=False)
print("Evaluation scores saved to 'transfer_combined_evaluation_scores.csv'.")

Evaluating model predicting 5 days ahead with 14 days timeframe.
9/9 [==============================] - 1s 119ms/step
Evaluation Metrics:
Accuracy: 0.7048611111111112
Precision: 0.9602649006622517
Recall: 0.6473214285714286
F1 Score: 0.7733333333333334
Hit Rate: 0.7048611111111112
Average Long RoR: 0.02122657212593089
Long Investment: 15100
Long Return: 320.5212391015564
Average Short RoR: -0.0007627115960907691
Short Investment: 13700
Short Return: -10.449148866443537
Total Investment: 28800
Total Return: 310.07209023511285
Evaluating model predicting 5 days ahead with 30 days timeframe.
9/9 [==============================] - 3s 335ms/step
Evaluation Metrics:
Accuracy: 0.5627240143369175
Precision: 0.8932038834951457
Recall: 0.45320197044334976
F1 Score: 0.6013071895424837
Hit Rate: 0.5627240143369175
Average Long RoR: 0.021479471994810963
Long Investment: 10300
Long Return: 221.23856154655292
Average Short RoR: -0.00474871502817351
Short Investment: 17600
Short Return: -83.5773844958

In [ ]:
# Train models on all images types and evaluate them based on timeframe and prediction

evaluation_df = pd.DataFrame(columns=['Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Average_RoR'])

for timeframe in TIMEFRAMES:
    for prediction in PREDICTIONS:
        if prediction < timeframe:
            print(f"Predicting {prediction} days ahead with {timeframe} days timeframe.")
            
            # Filter your data based on prediction, img_type, and timeframe
            data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(f'_{timeframe}_'))]
            data = data.reset_index(drop=True)  # Reset the index to maintain temporal order

            # Load model for timeframe and prediction
            model = load_model(f"models/combined_{timeframe}_{prediction}.h5")

            # Create a new array to store the combined predictions
            y_pred_combined = []
            y_combined = []

            for date in data['Date'].unique():
                for etf in NEW_DATA:
                    new_data = data[(data['Date']==date) & (data['Image'].str.contains(f'{etf}_'))]
                    new_data = data.reset_index(drop=True)
                
                    X = np.array(new_data['Image_Array'].tolist())/255.0
                    # y equals the first label in the DataFrame
                    y = new_data['Label'].iloc[0]
                    y_combined.append(y)
                    lastPrice = new_data['LastPrice'].iloc[0]
                    futurePrice = new_data['FuturePrice'].iloc[0]
                    
                    # Evaluate the model
                    y_pred = model.predict(X)
                    # Convert predictions to binary: if > 0.5 then 1 else 0
                    y_pred_binary = np.where(y_pred > 0.5, 1, 0)

                    # If y_pred_binary contains more 1s than 0s, then predict 1, else predict 0
                    if np.sum(y_pred_binary) > len(y_pred_binary) / 2:
                        y_pred_combined.append(1)
                    else:
                        y_pred_combined.append(0)
                
            accuracy = accuracy_score(y_combined, y_pred_combined)
            precision = precision_score(y_combined, y_pred_combined)
            recall = recall_score(y_combined, y_pred_combined)
            f1_score = fbeta_score(y_combined, y_pred_combined, beta=1)

            y_array = np.array(y_combined)
            correct_predictions = np.sum(np.array(y_pred_combined) == y_array)
            hit_rate = correct_predictions / len(y_array)

            # Calculate Rate of Return RoR
            ror = np.where(y_pred_combined == 1, ((futurePrice - lastPrice)/lastPrice), 0)
            # Drop all 0s from the array
            ror = ror[ror != 0]
            # Calculate average RoR
            avg_ror = np.mean(ror)
        
            print("Evaluation Metrics:")
            print(f"Accuracy: {accuracy}")
            print(f"Precision: {precision}")
            print(f"Recall: {recall}")
            print(f"F1 Score: {f1_score}")
            
            # Add the evaluation metrics to the DataFrame
            evaluation_df = evaluation_df.append({
                'Timeframe': timeframe,
                'Prediction': prediction,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1_Score': f1_score,
                'Hit_Rate': hit_rate,
                'Average_RoR': avg_ror
            }, ignore_index=True)
        
            print("--------------------------------------------------")

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('combined_prediction_evaluation_scores.csv', index=False)
print("Evaluation scores saved to 'combined_prediction_evaluation_scores.csv'.")

In [ ]:
ror_df = pd.DataFrame(columns=['Image_Type', 'Timeframe', 'Prediction', 'Average_RoR'])

for img_type in IMG_TYPES:
    for timeframe in TIMEFRAMES:
        for prediction in PREDICTIONS:
            if prediction < timeframe:

                # Filter your data based on prediction, img_type, and timeframe
                data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(img_type)) & (labels['Image'].str.contains(f'_{timeframe}_'))]
                data = data.reset_index(drop=True)
                split_index = int(len(data) * 0.7)
                lastPrice = data['LastPrice'][split_index:]
                futurePrice = data['FuturePrice'][split_index:]
                # Calculate Rate of Return RoR
                ror = np.array((futurePrice - lastPrice)/lastPrice)
                print(ror)
                # Calculate average RoR
                avg_ror = np.mean(ror)
                #print(f"Average RoR for {img_type} {timeframe} {prediction}: {avg_ror}")
                # Add the evaluation metrics to the DataFrame
                ror_df = ror_df.append({
                    'Image_Type': img_type,
                    'Timeframe': timeframe,
                    'Prediction': prediction,
                    'Average_RoR': avg_ror
                }, ignore_index=True)

# Save the evaluation DataFrame to a CSV file
ror_df.to_csv('transfer_ror.csv', index=False)

In [29]:
ror_df = pd.DataFrame(columns=['Timeframe', 'Prediction', 'Average_RoR'])

for timeframe in TIMEFRAMES:
    for prediction in PREDICTIONS:
        if prediction < timeframe:

            # Filter your data based on prediction, img_type, and timeframe
            data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(f'_{timeframe}_'))]
            data = data.reset_index(drop=True)
            split_index = int(len(data) * 0.7)
            lastPrice = data['LastPrice'][split_index:]
            futurePrice = data['FuturePrice'][split_index:]
            # Calculate Rate of Return RoR
            ror = np.array((futurePrice - lastPrice)/lastPrice)
            # Calculate average RoR
            avg_ror = np.mean(ror)
            print(f"Average RoR for {timeframe} {prediction}: {avg_ror}")
            # Add the evaluation metrics to the DataFrame
            ror_df = ror_df.append({
                'Timeframe': timeframe,
                'Prediction': prediction,
                'Average_RoR': avg_ror
            }, ignore_index=True)

# Save the evaluation DataFrame to a CSV file
ror_df.to_csv('transfer_ror.csv', index=False)

Average RoR for 14 5: 0.011599411435437365
Average RoR for 30 5: 0.013027316541640499
Average RoR for 90 5: 0.013107337888467137
Average RoR for 90 30: 0.013850716938016564
Average RoR for 180 5: 0.008278014057449852
Average RoR for 180 30: 0.011032570021406672
Average RoR for 180 90: 0.008131604052678987
Average RoR for 365 5: -4.523476409968202e-05
Average RoR for 365 30: -0.014622928083348707
Average RoR for 365 90: -0.02746736900707071
